## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,74.25,72,78,11.95,broken clouds
1,1,Sitka,US,57.0531,-135.3300,48.09,66,90,11.50,moderate rain
2,2,Sola,VU,-13.8833,167.5500,81.12,76,10,16.11,clear sky
3,3,Komsomolskiy,UZ,40.4272,71.7189,38.12,70,0,6.91,clear sky
4,4,Albany,US,42.6001,-73.9662,47.57,77,83,1.01,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= float(max_temp)) & (city_data_df["Max Temp"] >= float(min_temp))]

In [13]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Sola,VU,-13.8833,167.5500,81.12,76,10,16.11,clear sky
10,10,Faanui,PF,-16.4833,-151.7500,78.12,81,74,13.49,light rain
12,12,Busselton,AU,-33.6500,115.3333,75.04,58,99,6.35,overcast clouds
16,16,Hilo,US,19.7297,-155.0900,81.28,70,20,8.05,few clouds
20,20,Kapaa,US,22.0752,-159.3190,84.18,77,40,3.00,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
680,680,Loa Janan,ID,-0.5830,117.0950,86.88,64,97,3.56,overcast clouds
681,681,Cabadiangan,PH,9.7531,122.4739,84.00,74,100,7.23,overcast clouds
688,688,Dhidhdhoo,MV,6.8833,73.1000,81.55,78,100,11.83,light rain
689,689,Samarai,PG,-10.6167,150.6667,81.46,80,25,12.06,scattered clouds


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sola,VU,81.12,clear sky,-13.8833,167.5500,
10,Faanui,PF,78.12,light rain,-16.4833,-151.7500,
12,Busselton,AU,75.04,overcast clouds,-33.6500,115.3333,
16,Hilo,US,81.28,few clouds,19.7297,-155.0900,
20,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
...,...,...,...,...,...,...,...
680,Loa Janan,ID,86.88,overcast clouds,-0.5830,117.0950,
681,Cabadiangan,PH,84.00,overcast clouds,9.7531,122.4739,
688,Dhidhdhoo,MV,81.55,light rain,6.8833,73.1000,
689,Samarai,PG,81.46,scattered clouds,-10.6167,150.6667,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sola,VU,81.12,clear sky,-13.8833,167.5500,Leumerus Bungalows
10,Faanui,PF,78.12,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Busselton,AU,75.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
16,Hilo,US,81.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
680,Loa Janan,ID,86.88,overcast clouds,-0.5830,117.0950,Bangsal Pak Tikno
681,Cabadiangan,PH,84.00,overcast clouds,9.7531,122.4739,Kipot
688,Dhidhdhoo,MV,81.55,light rain,6.8833,73.1000,Dhidhoo beach
689,Samarai,PG,81.46,scattered clouds,-10.6167,150.6667,Nuli Sapi


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))